In [ ]:
#instalacion de dependecias

In [ ]:
!pip install dipy

In [ ]:
!pip install tensorflow

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers  import Add
from skimage.transform import rescale, resize, downscale_local_mean
from keras.layers import Input, Activation
from keras.models import Model
from keras import backend as K

from sklearn.model_selection import train_test_split
import tensorflow as tf


from keras.optimizers import Adam
from keras import losses

from skimage.metrics import structural_similarity as ssim
import cv2
import time
from tensorflow.keras.models import load_model
from google.colab import files




from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
import dipy.reconst.dti as dti
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from dipy.reconst.dti import fractional_anisotropy, color_fa
import dipy.data as dpd

In [ ]:
def plot_results_3_images(lowres, preds, highres):
  """
  mostrar las 3 imagenes(lowres, preds, highres)
  """
  i = 0
  fig, axes = plt.subplots(1, 3, figsize=(25,25))
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  ax1, ax2, ax3 = axes
  ax1.imshow(lowres)
  ax1.set_title('lowres')
  ax2.imshow(preds)
  ax2.set_title('prediction')
  ax3.imshow(highres)
  ax3.set_title('highres')

In [ ]:
#recurso: https://github.com/Likarian/VDSR-keras/tree/14a6bf94c7d23f197bcf0a18353bdcb4c7e3bd8c
def VDSR_origin(input_shape):
    low_resolution_image = Input(shape = input_shape )
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(low_resolution_image)    
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu')(processing)
    processing = Conv2D(1, (3, 3), padding='same', kernel_initializer='he_normal')(processing)
    Residual = processing

    high_resolution_image = Add() ([low_resolution_image, Residual])
    
    model = Model(low_resolution_image, high_resolution_image)
    return model

In [ ]:
#conexion a google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
interactive = True
filename = '/content/drive/MyDrive/ColabNotebooks/Data/TENSOR_2x2x2.nii' 
#se cargan las imagenes
img = nib.load(filename)
data = img.get_fdata()
#se obtiene el numero ttotal de imagenes
num_img = data.shape[3]*data.shape[2]

#se reduce la dimension de las imagenes
data = data.reshape(256,256,-1)
#se normalizan las imgenes
dataMax= data.max()
data = data/dataMax


In [ ]:
#se reduce la resolucion de las imagenes
scale = 2
dataLow=data[0:data.shape[0]:scale,0:data.shape[1]:scale,:]
dataLowR = tf.image.resize(dataLow, [256, 256], method='bicubic')
dataLowR=np.array(dataLowR)
print("LowRes Data resized",dataLowR.shape)

LowRes Data resized (256, 256, 858)


In [ ]:
#se transponen los ejes y se agrega una dismension extra porque es el formato que recibe la red
dataLowR,data = dataLowR.transpose(2,0,1),data.transpose(2,0,1)
dataLowR,data =dataLowR.reshape(-1,dataLowR.shape[1],dataLowR.shape[2],1),data.reshape(-1,data.shape[1],data.shape[2],1)
print("data shape:",data.shape)
print("datalowR shape:",dataLowR.shape)
#Se hace la separacion de datos es training y test teniendo una separacion de 85% y 15% respectivamente
trainX,testX,trainY,testY = train_test_split(dataLowR,data,test_size = 0.15,random_state = 2024)
print("TrainX shape:",trainX.shape)
print("TrainY shape:",trainY.shape)

data shape: (858, 256, 256, 1)
datalowR shape: (858, 256, 256, 1)
TrainX shape: (729, 256, 256, 1)
TrainY shape: (729, 256, 256, 1)


In [ ]:
# creacion, compilacion y entrenamiento del modelo
model = VDSR_origin(input_shape=(256, 256, 1))
opt = Adam(lr=3E-3, decay=1E-3)
model.compile(loss='mse', optimizer=opt, metrics=[losses.mse])

# Registro de tiempo de inicio
tiempo_inicio = time.time()
EPOCHS = 500
model_train = model.fit(trainX,trainY, batch_size=32, epochs=EPOCHS, validation_split=0.177)
# Registro de tiempo de finalización
tiempo_fin = time.time()

#grafica epocas vs Training loss y validation loss
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(EPOCHS)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
# Cálculo del tiempo total de ejecución y salida del resultado
tiempo_total = tiempo_fin - tiempo_inicio
print("Tiempo total de ejecución: ", tiempo_total, " segundos")#Tiempo total de ejecución:  50996.75665998459  segundos

model.save('modelVDSR500epochs.keras')
files.download("modelVDSR500epochs.keras")


In [ ]:

#metricas
def psnr(imagenesGT, imagenesPred):
  psnrGlobal = []
  for i in range(len(imagenesGT)):
    psnr = cv2.PSNR(imagenesGT[i], imagenesPred[i])
    psnrGlobal.append(psnr)
  psnrGlobal = np.array(psnrGlobal)
  return (psnrGlobal.mean(),psnrGlobal.std())


def rmse(imagenesGT, imagenesPred):
  rmseGlobal = []
  for i in range(len(imagenesGT)):
    rmse = np.sqrt(np.mean((imagenesGT[i] - imagenesPred[i])**2))
    rmseGlobal.append(rmse)
  rmseGlobal = np.array(rmseGlobal)
  return (rmseGlobal.mean(),rmseGlobal.std())

def mse(imagenesGT, imagenesPred):
  mseGlobal = []
  for i in range(len(imagenesGT)):
    mse = np.mean((imagenesGT[i] - imagenesPred[i])**2)
    mseGlobal.append(mse)
  mseGlobal = np.array(mseGlobal)
  return (mseGlobal.mean(),mseGlobal.std())

def ssim_(imagenesGT, imagenesPred):
  ssimGlobal = []
  for i in range(len(imagenesGT)):
    ssim_index = ssim(imagenesGT[i], imagenesPred[i], data_range=imagenesPred[i].max() - imagenesPred[i].min(), multichannel=True)
    ssimGlobal.append(ssim_index)
  ssimGlobal = np.array(ssimGlobal)
  return (ssimGlobal.mean(),ssimGlobal.std())


In [ ]:
#carga del modelo entrenado
#model = load_model('/content/drive/MyDrive/Tesis/modelVDSR500epochs.h5')

In [ ]:
#se definen las funciones de mascara
def create_mask(img):
  img = np.array(img)
  plt.imshow(img,cmap="gray")
  plt.show()
  print(img.dtype)
  img2 = img*255
  # Convertir la imagen a escala de grises
  gray = img2.astype("uint8")
  plt.imshow(gray,cmap="gray")
  plt.show()
  # Calcular el umbral Otsu para la imagen en escala de grises
  _,thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  plt.imshow(thresh,cmap="gray")
  plt.show()
 

  # Encontrar los contornos de la imagen binaria
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # Encontrar el contorno más grande
  max_contour = max(contours, key=cv2.contourArea)
  # Crear una imagen negra del mismo tamaño que la original
  mask = np.zeros_like(gray)

  # Dibujar el contorno más grande en blanco en la imagen negra
  cv2.drawContours(mask, [max_contour], 0, (255), -1)

  # Dilatar la máscara para ampliar el área de recorte
  kernel = np.ones((15, 15), np.uint8)
  mask = cv2.dilate(mask, kernel, iterations=1)

  return mask


def cut(img, mask):
  # Convertir la imagen a escala de grises
  gray = img.astype("uint8")
  # Aplicar la máscara a la imagen original
  masked_img = cv2.bitwise_and(img, img, mask=mask)

  return masked_img

def recortarSet(imagenes, mask):
  imagenes_recortadas = []
  for i in imagenes:
    imagenes_recortadas.append(cut(i, mask))
  return imagenes_recortadas

In [ ]:
#prediccion y recorte del conjunto de test
data = np.array(data)

ypred = model.predict(testX)
data = data.astype(np.float32)
testY = testY.astype(np.float32)


mask = create_mask(data[480])
data_cut = np.array(recortarSet(data, mask))
ypred_cut = np.array(recortarSet(ypred, mask))
testY_cut = np.array(recortarSet(testY, mask))
dataLowR_cut = np.array(recortarSet(dataLowR, mask))

In [ ]:
print("Metricas sin recorte")
valssim,stdssim = ssim_(testY, ypred)
print('SSIM:', valssim, "standard deviation: ", stdssim)
valmse,stdmse = mse(testY, ypred)
print('MSE:', valmse, "standard deviation: ", stdmse)
valrmse,stdrmse = rmse(testY, ypred)
print('RMSE:', valrmse, "standard deviation: ", stdrmse)
valpsnr,stdpsnr = psnr(testY, ypred)
print('PSNR:', valpsnr, "standard deviation: ", stdpsnr)

print("Metricas con recorte")
valssim,stdssim = ssim_(testY_cut, ypred_cut)
print('SSIM:', valssim, "standard deviation: ", stdssim)
valmse,stdmse = mse(testY_cut, ypred_cut)
print('MSE:', valmse, "standard deviation: ", stdmse)
valrmse,stdrmse = rmse(testY_cut, ypred_cut)
print('RMSE:', valrmse, "standard deviation: ", stdrmse)
valpsnr,stdpsnr = psnr(testY_cut, ypred_cut)
print('PSNR:', valpsnr, "standard deviation: ", stdpsnr)

In [ ]:
def backTo4D(img):
  """
  Funcion que dado un conjunto de imagnes con dimensiones (858,256,256)
  lo convierte a (256,256,33,26)
  """
  nimg =  img.reshape(858,256,256)
  nimg = nimg.transpose(1,2,0)
  nimg = nimg.reshape(256,256,33,26)
  return nimg

In [ ]:
#se leen los valores propios y vectores propios de los tensores
bvals, bvecs = read_bvals_bvecs('/content/drive/MyDrive/ColabNotebooks/Data/TENSOR_2x2x2.bval', '/content/drive/MyDrive/ColabNotebooks/Data/TENSOR_2x2x2.bvec')

data_cut4 = backTo4D(data_cut)
gtab = gradient_table(bvals, bvecs)
tensorm = dti.TensorModel(gtab)
tenfitdata = tensorm.fit(data_cut4)

In [ ]:
dataLowpred = model.predict(dataLowR)
dataLow_cut = np.array(recortarSet(dataLowpred, mask))

dataLow4 = backTo4D(dataLow_cut)
gtab = gradient_table(bvals, bvecs)
tensorm = dti.TensorModel(gtab)
tenfitdatalow = tensorm.fit(dataLow4)

27/27 [==============================] - 16s 613ms/step


In [ ]:
#se calcula la metrica de norma de frobenius
fnorm = dti.norm(tenfitdata.quadratic_form - tenfitdatalow.quadratic_form)
print("mean: ",fnorm.mean(), " std: ", fnorm.std())

mean:  0.00015885129904424965  std:  0.0003825490417568843


In [ ]:
#se calcula la metrica de MSE de FA
FAdata = tenfitdata.fa.transpose(2,0,1)
FApred = tenfitdatalow.fa.transpose(2,0,1)
valfa,stdfa = mse(FAdata, FApred)
print('MSE de FA:', valfa, "standard deviation: ", stdfa)

(256, 256, 33)
MSE de FA: 0.01552567431065916 standard deviation:  0.009817935583266775
